## Importing

In [ ]:
import pandas as pd
import numpy as np

# try to import TA-Lib; fallback flag if unavailable
try:
    import ta-lib
    TALIB_AVAILABLE = True
except ImportError:
    TALIB_AVAILABLE = False
    print("TA-Lib not available; falling back to pure-Python implementations.")

## Class Declaration

In [ ]:
class QuantitativeAnalysis:
    def __init__(self, stock_df: pd.DataFrame):
        self.df = stock_df.copy()
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.df.sort_values('Date', inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        self.df['daily_return'] = self.df['Close'].pct_change()

    def add_sma(self, window=20):
        if TALIB_AVAILABLE:
            self.df[f"SMA_{window}"] = talib.SMA(self.df['Close'], timeperiod=window)
        else:
            self.df[f"SMA_{window}"] = self.df['Close'].rolling(window).mean()

    def add_rsi(self, period=14):
        if TALIB_AVAILABLE:
            self.df['RSI'] = talib.RSI(self.df['Close'], timeperiod=period)
        else:
            delta = self.df['Close'].diff()
            up = delta.clip(lower=0)
            down = -1 * delta.clip(upper=0)
            ma_up = up.rolling(period).mean()
            ma_down = down.rolling(period).mean()
            rs = ma_up / (ma_down + 1e-8)
            self.df['RSI'] = 100 - (100 / (1 + rs))

    def add_macd(self):
        if TALIB_AVAILABLE:
            macd, signal, hist = talib.MACD(self.df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
            self.df['MACD'] = macd
            self.df['MACD_signal'] = signal
            self.df['MACD_hist'] = hist
        else:
            # simple MACD approximation using EMAs
            ema12 = self.df['Close'].ewm(span=12, adjust=False).mean()
            ema26 = self.df['Close'].ewm(span=26, adjust=False).mean()
            self.df['MACD'] = ema12 - ema26
            self.df['MACD_signal'] = self.df['MACD'].ewm(span=9, adjust=False).mean()
            self.df['MACD_hist'] = self.df['MACD'] - self.df['MACD_signal']

    def summary(self):
        return self.df.tail()

    def plot_price_with_sma(self):
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,5))
        plt.plot(self.df['Date'], self.df['Close'], label='Close')
        for col in self.df.columns:
            if col.startswith('SMA_'):
                plt.plot(self.df['Date'], self.df[col], label=col)
        plt.title("Price with SMAs")
        plt.legend()
        plt.tight_layout()
        plt.show()

## Analysis

In [ ]:
# load AAPL (adjust path as needed)
aapl_df = pd.read_csv("../data/AAPL_historical_data.csv")

qa = QuantitativeAnalysis(aapl_df)
qa.add_sma(20)
qa.add_sma(50)
qa.add_rsi()
qa.add_macd()

qa.plot_price_with_sma()
display(qa.summary())